# Introduction
-----------------------------

In this notebook I'm using [Pantheon](https://github.com/Canisback/pantheon) to define various other functions for using the Riot API. The reason for using Pantheon is to make use of it's rate limiting system, since as a standalone account I can only make so many calls to the API without getting blacklisted.

In [1]:
import requests, json, csv, asyncio, nest_asyncio, datetime
import pandas as pd
from pantheon import pantheon